<a href="https://colab.research.google.com/github/ahembal/MetinMadenciligiLec/blob/main/Homework_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connectting to drive to read txt file:


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [13]:
ls

 Baskaldiran-Insan-Albert-Camus.txt   Homework-01.ipynb     Untitled1.ipynb
'Copy of language_modelling.ipynb'    RNNsStillCool.ipynb
'Copy of word_embeddings.ipynb'       Untitled0.ipynb


## Part 0
################################################################################
# Part 0: Utility Functions
################################################################################

In [1]:
import math, random



COUNTRY_CODES = ['af', 'cn', 'de', 'fi', 'fr', 'in', 'ir', 'pk', 'za']

def start_pad(n):
    ''' Returns a padding string of length n to append to the front of text
        as a pre-processing step to building n-grams '''
    return '~' * n

def ngrams(n, text):
    ''' Returns the ngrams of the text as tuples where the first element is
        the length-n context and the second is the character '''
    padded_tokens = []
    padded_tokens = "~"*(n) + text + "."
    if n == 1:
      lst = [((padded_tokens[i-1]), token) for i, token in enumerate(padded_tokens) if i >= n-1]    # >>> ngrams(1, 'abc') => [('~', 'a'), ('a', 'b'), ('b', 'c')]
      return  lst[1:]
    else:
      lst = [(("".join(padded_tokens[i-n:i])), token) for i, token in enumerate(padded_tokens) if i >= n-1] # >>> ngrams(2, 'abc') => [('~~', 'a'), ('~a', 'b'), ('ab', 'c')]
      return  lst[1:]

def create_ngram_model(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        model.update(f.read())
    return model

def create_ngram_model_lines(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        for line in f:
            model.update(line.strip())
    return model

## Part 1
################################################################################
# Part 1: Basic N-Gram Model
################################################################################

In [19]:


class NgramModel():
    ''' A basic n-gram model using add-k smoothing '''

    def __init__(self, n, k):
        self.n = n
        self.k = k
        self.context_dic = {}
        self.context_count_dic = {}

    def get_vocab(self):
        ''' Returns the set of characters in the vocab '''
        return self.context_count_dic.keys()

    def update(self, text):
        ''' Updates the model n-grams based on text '''
        for context, token in ngrams(self.n, text):
            # keep count
            if context in self.context_count_dic:
                self.context_count_dic[context] += 1
            else:
                self.context_count_dic[context] = 1

            # insert data
            if context in self.context_dic:
                token_dic = self.context_dic.get(context)
                if token in token_dic:
                    token_dic[token] += 1
                else:
                    token_dic[token] = 1
            else:
                self.context_dic[context] = {token: 1}
        return

    def prob(self, context, char):
        ''' Returns the probability of char appearing after context '''
        if context in self.context_dic:
            token_dic = self.context_dic[context]
            if char in token_dic:
                return float(token_dic[char]) / self.context_count_dic[context]
            else:
                return 0.001
        else:
            return 0.001

    def random_char(self, context):
        ''' Returns a random character based on the given context and the 
            n-grams learned by this model '''
        r = random.random()

        if context in self.context_dic:
            denominator = self.context_count_dic[context]
            token_dic = self.context_dic[context]
            sorted_keys = sorted(token_dic.keys())

            for i, token in enumerate(sorted_keys):
                minus_i_sum = sum([token_dic[k] for k in sorted_keys[:i]])
                if float(minus_i_sum)/denominator <= r < float(minus_i_sum + token_dic[sorted_keys[i]])/denominator:
                    return token

        else:
            return None

    def random_text(self, length):
        ''' Returns text of the specified character length based on the
            n-grams learned by this model '''
        if self.n != 1:
            context = "".join(["~"] * (self.n))
            generated = []

            for __ in range(length):
                token = self.random_char(context)
                generated.append(token)

                if token != ".":
                    context = context[1:] + (token)
                else:
                    context = "".join(["~"] * (self.n))

            return "".join(generated)
        else:
            return "".join(self.random_char(random.choice(list(self.get_vocab()))) for _ in range(length))

    def perplexity(self, text):
        ''' Returns the perplexity of text based on the n-grams learned by
            this model '''
        product = 0

        for context, token in ngrams(self.n, text):
          if self.prob(context,token) != 0:
            product += math.log(self.prob(context, token))
        return (1/math.exp(product)) ** (float(1)/(len(text)+1))

## Test Part

### 1 Basic n-gram Models



In [38]:
n = NgramModel(1,0)
n.update('abab')

In [41]:
print(n.perplexity('ab'))
print(n.perplexity('cd'))

1.2599210498948732
999.9999999999994


In [42]:
n.update('cd')
n.perplexity('cd')

1.2599210498948732

In [24]:
if __name__ == '__main__':
  model1 = create_ngram_model(NgramModel, "Baskaldiran-Insan-Albert-Camus.txt", n=1)
  model2 = create_ngram_model(NgramModel, "Baskaldiran-Insan-Albert-Camus.txt", n=2)
  model3 = create_ngram_model(NgramModel, "Baskaldiran-Insan-Albert-Camus.txt", n=3)
  model4 = create_ngram_model(NgramModel, "Baskaldiran-Insan-Albert-Camus.txt", n=4)
  model5 = create_ngram_model(NgramModel, "Baskaldiran-Insan-Albert-Camus.txt", n=5)
  model6 = create_ngram_model(NgramModel, "Baskaldiran-Insan-Albert-Camus.txt", n=6)
  model7 = create_ngram_model(NgramModel, "Baskaldiran-Insan-Albert-Camus.txt", n=7)
  model10 = create_ngram_model(NgramModel, "Baskaldiran-Insan-Albert-Camus.txt", n=10)
  model15 = create_ngram_model(NgramModel, "Baskaldiran-Insan-Albert-Camus.txt", n=15)




  print('============= Uni-gram =============')

  #print(model1.get_vocab())
  #print('-------')
  print(model2.random_text(300))


  print()
  print('============= Bi-gram =============')

  #print('-------')
  print(model2.random_text(300))


  print()
  print('============= Tri-gram =============')

  print(model3.random_text(300))


  print()
  print('============= Four-gram =============')

  print(model4.random_text(300))



  print()
  print('============= Five-gram =============')

  print(model5.random_text(300))


  print()
  print('============= Six-gram =============')

  print(model6.random_text(300))

  print()
  print('============= Seven-gram =============')

  print(model7.random_text(300))


  print()
  print('============= Ten-gram =============')

  print(model10.random_text(300))


  print()
  print('============= 15-gram =============')

  print(model15.random_text(300))

============= Uni-gram =============
ALBEL Yaz, sün etizliklarx’ını birekmış düne sürmekişin böylendirde, l’e sözelerin ve de derçekmadırlüğü bu kalaraktu.ALTIK Kitiminek sonu bilirimir bulununları varçeğilerin kur, isinini bi, büyüzeyecekla dalışı olanınını eni ve (Konursledeği olarışılarış biganı düğü, bütülem olamak içimektık ta nay

============= Bi-gram =============
ALDIRI Ver.ALDÜLEME


22.ALDIRICILDIRAN

65 yazlıdanınaylabil, sürür kalı bilinide, sarderksindaynı.AL Yen Cebilerluğun Salyayaşkulurjuvartına deme yan eğin keninsayısındi.AL Yar ile, der, aşı am basın du.ALTIK Ve değ bır hak Tahaklaralıkım derileydı.ALDIRI

Sonundir bilin, kirlarşılamamazlunu denda

============= Tri-gram =============
ALBERT CAMUS

Böylemek belişmez, Tanrı örne, yargılaşmazlamı söylemiş olmayıcı gize.ALBERT CAMUS

Bir şeyi ve ama tan yollanmıştır.ALBERT CAMUS

Bakunur.ALBERT CAMUS, 1789 yen san bütününü söyle çevreniden bir şey uymaz aral Söyle başlı öldürmeye savaşı de konun emi bir bileri yılığı b

In [16]:
model1.get_vocab()

dict_keys(['~', 'A', 'L', 'B', 'E', 'R', 'T', ' ', 'C', 'M', 'U', 'S', '\n', 'Ş', 'K', 'D', 'I', 'N', 'İ', '1', '9', '5', '7', 'O', 'Y', 'Ö', 'Ü', 'F', 'r', 'a', 'n', 's', 'ı', 'z', 'c', 'l', 'd', 'ç', 'e', 'v', 'i', 'h', 'ü', 'b', 't', 'm', 'u', '’', 'y', 'k', 'ğ', 'p', ':', ',', '8', '2', 'o', 'G', '3', '4', '-', 'g', '6', 'f', 'ö', 'ş', 'V', '.', 'P', '“', '”', 'X', '0', 'H', '(', ')', 'J', 'x', 'é', 'Ç', 'q', 'â', ';', 'Z', '*', '?', '–', '!', 'î', 'j', 'Ğ', 'û', 'è', 'ê', 'w', '/', 'Q', 'W', '‘', 'Â'])

In [35]:
print(model1.prob('A','l'))
print(model1.random_char('A'))
print(model1.perplexity("Yaz"))

0.10068649885583524
m
39.25554000250682
